### Attempt to make, new improved x-ray structure analyser


Could do an initial proximity check to see which residues are nearby each other, then only focus on those for the analysis?


Notes: If among standard 20 amino acids, can split into bb and side chain.

If not, then no split is possible but can be treated as a ligand.

Can also provide option to either provide, 1 or 0, or some scaled value...



Hbond defintion:
- up till 3.5 A, reducing between 2 and 3.5 good.
- 180 +- xxx degrees, towards 180 =better.
- Make a model that swaps between 0 and 1 for this? 


vdW defintion. 
- Can do as before, but do last and only if something between them already not defined. 



In [2]:
from typing import Tuple, Optional, List
import warnings
import itertools
import pandas as pd
import numpy as np


import MDAnalysis
from MDAnalysis import Universe
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
from MDAnalysis.analysis import contacts # TODO perhaps remove and use distances implementation?
from MDAnalysis.analysis import distances

/Users/dariiayehorova/miniforge3/envs/tools_proj_py3_11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/dariiayehorova/miniforge3/envs/tools_proj_py3_11/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [3]:
# Amino acid definitions - helps define interaction types.
POSITIVE_SB_RESIDUES = ("LYS", "ARG")
NEGATIVE_SB_RESIDUES = ("GLU", "ASP")
HYDROPHOBIC_RESIDUES = ("ALA", "VAL", "LEU", "ILE", "PRO", "PHE", "CYS")
# CATION PI
# AROMATIC 
# OTHER INTERACTION TYPES. 
STANDARD_20_RESIDUES = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLU", "GLN", "GLY", 
                        "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", 
                        "THR", "TRP", "TYR", "VAL"]


In [4]:
parm_file = r"WT_PTP1B_Closed.pdb" # r"PDZ_Apo.pdb"
traj_file = r"WT_PTP1B_Closed.pdb" # r"PDZ_Apo.pdb"
universe = Universe(parm_file, traj_file)

/Users/dariiayehorova/miniforge3/envs/tools_proj_py3_11/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:331: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "


In [5]:
res_names = universe.residues.resnames
print(res_names[0:5])

['GLU' 'MET' 'GLU' 'LYS' 'GLU']


In [6]:
res1, res2 = 28, 73
res1_bb_sele = "name C CA O N H and resid " + str(res1)
res1_sc_sele = "not name C CA O N H and resid " + str(res1)
res1_bb_atoms = universe.select_atoms(res1_bb_sele)
res1_sc_atoms = universe.select_atoms(res1_sc_sele)

res2_bb_sele = "name C CA O N H and resid " + str(res2)
res2_sc_sele = "not name C CA O N H and resid " + str(res2)
res2_bb_atoms = universe.select_atoms(res2_bb_sele)
res2_sc_atoms = universe.select_atoms(res2_sc_sele)

res1_bb_atoms, res1_sc_atoms, res2_bb_atoms, res2_sc_atoms 

(<AtomGroup with 5 atoms>,
 <AtomGroup with 7 atoms>,
 <AtomGroup with 5 atoms>,
 <AtomGroup with 12 atoms>)

In [7]:
bb1_bb2_dists = contacts.distance_array(
    res1_bb_atoms.positions, res2_bb_atoms.positions
)
bb1_bb2_dists 

array([[16.90094717, 17.50930785, 17.01616876, 18.07523556, 19.26726043],
       [16.98084045, 17.61729793, 17.06851731, 18.07232127, 19.27653506],
       [17.74663463, 18.32236559, 17.95738583, 19.04662139, 20.22863419],
       [17.60548782, 18.19188767, 17.88842286, 18.91988226, 20.1081985 ],
       [17.95747147, 18.57999519, 18.20380095, 19.16105754, 20.36419746]])

In [8]:
for atom in res1_bb_atoms:
    print(atom)

<Atom 431: N of type N of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 432: H of type H of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 433: CA of type C of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 441: C of type C of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 442: O of type O of resname ASP, resid 28 and segid SYSTEM and altLoc >


In [9]:
for atom in res1_bb_atoms:
    print(atom)

<Atom 431: N of type N of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 432: H of type H of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 433: CA of type C of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 441: C of type C of resname ASP, resid 28 and segid SYSTEM and altLoc >
<Atom 442: O of type O of resname ASP, resid 28 and segid SYSTEM and altLoc >


### Salt bridges

Could just do an electrostatics based calculation? Need force field charges though...

In [10]:
from tools_proj.contacts.salt_bridges import check_for_salt_bridge
check_for_salt_bridge(res_numbers=(64, 55), universe=universe)

'ASP64 ARG55 saltbridge sc-sc'

In [11]:
universe.residues

<ResidueGroup with 298 residues>

In [12]:
for res1 in range(1, 298+1):
    for res2 in range(res1, len(universe.residues) + 1): 
        result = check_for_salt_bridge(res_numbers=(res1, res2), universe=universe)
        if result:
            print(result)

GLU1 LYS4 saltbridge sc-sc
GLU5 LYS246 saltbridge sc-sc
ASP10 ARG267 saltbridge sc-sc
ARG55 ASP64 saltbridge sc-sc
GLU75 ARG237 saltbridge sc-sc
ARG78 ASP228 saltbridge sc-sc
ARG104 GLU169 saltbridge sc-sc
GLU114 ARG220 saltbridge sc-sc
LYS119 ASP180 saltbridge sc-sc
GLU146 ARG155 saltbridge sc-sc
LYS149 ASP297 saltbridge sc-sc
ASP180 ARG220 saltbridge sc-sc
ARG198 ASP228 saltbridge sc-sc
ASP235 LYS238 saltbridge sc-sc
GLU251 LYS254 saltbridge sc-sc
GLU275 LYS278 saltbridge sc-sc


Done and works, could also consider a sum of distances to measure total strength.

### Cation-Pi interactions

Using this for definitions: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8338773/ 


In [12]:
from tools_proj.contacts.cation_pi import check_for_cation_pi
check_for_cation_pi(res_numbers=(99, 168), universe=universe)

'TRP99 ARG168 CationPi SC-SC'

In [13]:
for res1 in range(1, 298+1):
    for res2 in range(res1, len(universe.residues) + 1): 
        result = check_for_cation_pi(res_numbers=(res1, res2), universe=universe)
        if result:
            print(result)

TRP99 ARG168 CationPi SC-SC


Happy with this :) 

### Pi-Pi Interactions
Include stacking and t-shaped together 

Should include histidines also it seems. 

Referance for defintions:
https://link.springer.com/article/10.1007/s12539-015-0263-z 

In [14]:
# Constants
# From souce, 5 membered rings used for pi-pi stacking for TRP. 
PI_PI_RES_ATOMS = {"TYR": "name CG CE1 CE2", "PHE": "name CG CE1 CE2", 
                   "TRP": "name CG CD2 NE1", "HIE": "name CG CD2 CE1",
                   "HID": "name CG CD2 CE1"} # HIP not included as positive

# cutoffs
PI_PI_D_CUT = 7.2
PI_PI_ANGLE_TOLERANCE = 30 # how close to ideal the angle needs to be.



# Ideal values
PI_PI_DELTA_DISALLOWED = (-30, 30) # range of values NOT allowed# TODO double check. 

PI_PI_GAMMA_IDEAL_STACK = (-30, 30) # range of values allowed.
PI_PI_GAMMA_IDEAL_TSHAPED = (50, 130) # range of values allowed. 


In [15]:
#  15, 270 = t-shaped
res1_numb, res2_numb = 15, 270 # 94, 123 # stacked ish...
res1_sc_atoms = universe.select_atoms("not name C CA O N H and resid " + str(res1_numb))
res2_sc_atoms = universe.select_atoms("not name C CA O N H and resid " + str(res2_numb))
res1_name, res2_name = res1_sc_atoms.resnames[0], res2_sc_atoms.resnames[0]
res1_name, res2_name

('TRP', 'TYR')

In [16]:
if res1_name and res2_name not in PI_PI_RES_ATOMS:
    # return None
    print("would be skipped") 
res1_pi_atoms = res1_sc_atoms.select_atoms(PI_PI_RES_ATOMS[res1_name])
res2_pi_atoms = res2_sc_atoms.select_atoms(PI_PI_RES_ATOMS[res2_name])

In [17]:
# distance check.
com_dist = distances.distance_array(res1_pi_atoms.center_of_mass(), res2_pi_atoms.center_of_mass(), box=universe.dimensions)
if com_dist > PI_PI_D_CUT: # too far away
    # return None
    print("would be skipped")

In [18]:
# gamma (y) check
# seems to be vector of normal of res1 ring and normal of res2 ring. 
from tools_proj.contacts.utils import angle_between_two_vectors, normal_vector_3_atoms
res1_normal_vector = normal_vector_3_atoms(res1_pi_atoms.positions)
res2_normal_vector = normal_vector_3_atoms(res2_pi_atoms.positions)
gamma = angle_between_two_vectors(res1_normal_vector, res2_normal_vector)
gamma

103.70942177234181

In [19]:
if (gamma >= PI_PI_GAMMA_IDEAL_TSHAPED[0]) and (gamma <= PI_PI_GAMMA_IDEAL_TSHAPED[1]):
    print("t-shaped")

if (gamma >= PI_PI_GAMMA_IDEAL_STACK[0]) and (gamma <= PI_PI_GAMMA_IDEAL_STACK[1]):
    print("Stacked")

# TODO - add one more test... 

# TODO - from paper, Fig 4b, there are more spurois interactions to remove.... 


# if none, then continue... 

t-shaped


### (Side Chain) Hbonds

Can have a protocol if topology provided, but otherwise, this will work with standard 20 amino acids 
Could the expansion just be that I add the backbone names to each one, and then select from the whole aa and not from the side chain? 
Once a hydrogen bond found, can label it based on the atoms used perhaps?

In [20]:
SC_HBOND_RES_ATOMS_ACCEPTOR = {"ASN": ["OD1"], 
                                "GLN": ["OE1"], 
                                "SER": ["OG"], 
                                "THR": ["OG1"], 
                                "TYR": ["OH"], 
                                "GLU": ["OE1", "OE2"], 
                                "GLU": ["OE1", "OE2"],
                                "ASP": ["OD1", "OD2"],
                                "ASH": ["OD1", "OD2"],
                                "HIE": ["ND1"],
                                "HID": ["NE2"]
}
# List of possible combinations? 
# Hydrogens that match the pair? 
SC_HBOND_RES_ATOMS_DONOR = {"ASN": [{"ND2": ["HD21", "HD22"]}], 
                            "ARG": [{"NE": ["HE"]}, {"NH1": ["HH11", "HH12"]}, {"NH2": ["HH21", "HH22"]}], 
                            "GLN": [{"NE2": ["HE21", "HE22"]}],
                            "SER": [{"OG": ["HG"]}],
                            "THR": [{"OG1": ["HG1"]}],
                            "TYR": [{"OH": ["HH"]}], 
                            "HIE": [{"NE2": ["HE2"]}], 
                            "HID": [{"ND1": ["HD1"]}], 
                            "HIP": [{"NE2": ["HE2"]}, {"ND1": ["HD1"]}],  
                            "LYS": [{"NZ": ["HZ1", "HZ2", "HZ3"]}], 
                            "LYN": [{"NZ": ["HZ2", "HZ3"]}]
}

# cutoffs. 
HB_DIST_CUTOFF = 3.5 # Donor-acceptor Distance
HB_ANGLE_IDEAL = 180 
HB_ANGLE_TOLERANCE = 45 #

In [21]:
res1, res2 = 39, 63 # 39, 63 # Hbond, 
res1_bb_sele = "name C CA O N H and resid " + str(res1)
res1_sc_sele = "not name C CA O N H and resid " + str(res1)
res1_bb_atoms = universe.select_atoms(res1_bb_sele)
res1_sc_atoms = universe.select_atoms(res1_sc_sele)

res2_bb_sele = "name C CA O N H and resid " + str(res2)
res2_sc_sele = "not name C CA O N H and resid " + str(res2)
res2_bb_atoms = universe.select_atoms(res2_bb_sele)
res2_sc_atoms = universe.select_atoms(res2_sc_sele)

res1_name, res2_name = res_names[res1-1], res_names[res2-1]
res1_name, res2_name

('ASN', 'ASN')

In [22]:
# Need to consider that if and elif can both be true, and need to test both! 


if (res1_name in SC_HBOND_RES_ATOMS_ACCEPTOR.keys()) and (res2_name in SC_HBOND_RES_ATOMS_DONOR.keys()):
    hb_potential_acceptor_atoms = SC_HBOND_RES_ATOMS_ACCEPTOR[res1_name]
    hb_potential_donor_h_pairs = SC_HBOND_RES_ATOMS_DONOR[res2_name]

    combinations = list(itertools.product(hb_potential_acceptor_atoms, hb_potential_donor_h_pairs))

    for acceptor, donor in combinations:
        acceptor_atom = res1_sc_atoms.select_atoms("name " + acceptor)
        donor_atom = res2_sc_atoms.select_atoms("name " + list(donor.keys())[0])

        # remove donor_acceptor that fail distance test. 
        d_a_dist = distances.distance_array(acceptor_atom.positions, donor_atom.positions)
        if d_a_dist > HB_DIST_CUTOFF:
            continue

        for hydrogen in list(donor.values())[0]:
            hydrogen_atom = res2_sc_atoms.select_atoms("name " + hydrogen)

            dh_vector = donor_atom.positions[0] - hydrogen_atom.positions[0]
            ha_vector = acceptor_atom.positions[0] - hydrogen_atom.positions[0] 
            dha_angle = angle_between_two_vectors(dh_vector, ha_vector) 

            delta_dha_ideal = min(
                np.abs(dha_angle - HB_ANGLE_IDEAL),
                np.abs(dha_angle - HB_ANGLE_IDEAL - 180)
            )
            if delta_dha_ideal < HB_ANGLE_TOLERANCE:
                print("H-bond found!")
                print(acceptor_atom.names, donor_atom.names, hydrogen_atom.names)

elif (res1_name in SC_HBOND_RES_ATOMS_DONOR.keys()) and (res2_name in SC_HBOND_RES_ATOMS_ACCEPTOR.keys()):
    hb_potential_donor_h_pairs = SC_HBOND_RES_ATOMS_DONOR[res1_name]
    hb_potential_acceptor_atoms = SC_HBOND_RES_ATOMS_ACCEPTOR[res2_name]
    # TODO, logic would be here, for follow up.
    # test_sc_hbond()

else: 
    print("no hbonds possible.")

H-bond found!
['OD1'] ['ND2'] ['HD21']


### Hydrophobic

Requirements:
- 4 Å min distance.
- Between two "hydrophobic" residues 
- The atoms that contact must be non-polar (e.g. carbon).
- The atom should also be from the side-chain.

In [5]:
HYDROPHOBIC_RESIDUES = ("ALA", "VAL", "LEU", "ILE", "PRO", "PHE", "CYS")
HYDROPHOBIC_CUTOFF_DIST = 4.00


In [8]:
protein_atoms = universe.select_atoms("protein")
residue_indices=[]
for name in HYDROPHOBIC_RESIDUES:
    residue_indices.append(protein_atoms.residues.resids[protein_atoms.residues.resnames == name])
residue_indices=np.concatenate(residue_indices)
print(residue_indices)
side_chain_atoms = protein_atoms.select_atoms("not backbone")
counter=0

for res1 in residue_indices:
    res1_numb = res1 
    res1_name = universe.residues.resnames[res1_numb-1]
    res1_sc_sele = " type C and not backbone and resid " + str(res1)
    res1_sc_atoms = universe.select_atoms(res1_sc_sele)
    residue_indices = residue_indices[residue_indices != res1]
        
    for res2 in residue_indices:
        res2_numb = res2 
        res2_name = universe.residues.resnames[res2_numb-1] 
        res2_sc_sele = " type C and not backbone and resid " + str(res2)
        res2_sc_atoms = universe.select_atoms(res2_sc_sele)
        dist = distances.distance_array(
                res1_sc_atoms.positions,
                res2_sc_atoms.positions,
                box=universe.dimensions
                )
        if dist.min() <= HYDROPHOBIC_CUTOFF_DIST:
            print(res1_name + str(res1_numb) + " " + res2_name + str(res2_numb) + " Hydrophobic interaction")
            counter +=1
print("number of hydrophobic interactions:", counter)


[ 16  17  26  34  68  76 121 188 216 227 263 272 277  33  48  98 106 107
 112 154 183 197 210 211 212 243 248 273 286  36  58  70  82  87 109 118
 139 141 143 157 159 162 171 191 194 203 226 231 232 233 249 250 259 266
 271 293 298   9  18  22  56  66  71  81 133 144 148 170 218 245 260 274
 280  30  37  50  86  88 125 179 184 187 205 209 240   6  29  51  94 134
 173 181 190 193 195 224 255 268 279  31  91 120 225 230]
ALA34 PRO50 Hydrophobic interaction
ALA76 LEU233 Hydrophobic interaction
ALA188 LEU293 Hydrophobic interaction
ALA227 PHE190 Hydrophobic interaction
ALA272 PHE190 Hydrophobic interaction
VAL48 ILE218 Hydrophobic interaction
VAL98 LEU159 Hydrophobic interaction
VAL98 ILE170 Hydrophobic interaction
VAL98 PHE94 Hydrophobic interaction
VAL106 PHE94 Hydrophobic interaction
VAL107 VAL210 Hydrophobic interaction
VAL107 LEU109 Hydrophobic interaction
VAL107 PHE193 Hydrophobic interaction
VAL112 LEU118 Hydrophobic interaction
VAL154 PHE173 Hydrophobic interaction
VAL183 PRO184 Hy

In [ ]:
def hydrophobic_contacts():
    

#### Metals?
